# 图像分类网络之ConvNeXt

## 模型简介

ConvNeXt网络由Facebook AI研究所和UC Berkeley大学共同提出，它是一个面向2020s年代的卷积神经网络模型，并在论文[A ConvNet for the 2020s](https://arxiv.org/abs/2201.03545)中首次对其进行描述。

ConvNeXt网络并没有在整体的网络框架和搭建思路上做重大的创新，它主要是按照Transformer网络的一些思想对现有的经典ResNet网络做了一些改进，该网络在多个分类任务和识别任务中均超越了Swin-T模型达到最佳的性能表现。其中在ImageNet-1K的分类任务中，ConvNext网络与其他经典网络相比，性能如下图所示。

<div align=center><img src="./images/figure1.png"></div>

图中，左边ImageNet-1K的分辨率为224，右边ImageNet-1K的分辨率为384。由上图1也可发现ConvNext网络将2010年代的网络精度进行了提升，因此论文将所提出的ConvNext称为“2020年代的卷积网络”。

### 网络特点

下图2概括了ConvNext所有的优化点，同时图2给出了每一个优化点对网络精度以及FLOPs的影响。它从ResNet-50/ResNet-200出发，依次从宏观设计（Macro design）、深度可分离卷积（ResNeXt）、逆瓶颈层（Inverted bottleneck）、大卷积核（Large kernel size）以及细节设计（Various layer-wise Micro designs）这五个方面依次借鉴了Swin Transformer的思想，然后在ImageNet-1K上进行了训练和评估，由下图可发现，在相同的FLOPs下，相较于Swin TransFormer，ConvNext-T/B精度提升了0.7。

<div align=center><img src="./images/figure2.png"></div>

接下来依次介绍改动的五个部分，但在详细介绍每个部分之前，先介绍一下训练方法上的改进。

0. 训练技巧

论文采用与Swin Transformer相似的训练方法，大致从以下四个方法进行改进：（1）epoch从ResNet的90增加到300；（2）优化器从SGD改用AdamW优化器；（3）在数据增强方面引进Mixup、Cutmix、RandAugment和Random Erasing等；（4）增加正则化策略，例如使用随机深度、标签平滑、EMA等。更加具体的预训练和微调技巧的超参数如下图。

<div align=center><img src="./images/train1.jpg"></div>

采用这些训练技巧后，ResNet-50的性能提升了2.7%，从76.1%提升到78.8%。该结果证明，传统卷积与Transformer之间的性能差异也有一部分来自于训练的方法。

接下来对模型本身五大优化点进行详细的介绍。

1. Macro design

宏观设计部分主要有两项改动：

（1）stage比例： ResNet和Swin-T网络均有四个stage阶段，其中Swin-T各个阶段堆叠Block块的比例为1:1:3:1，Swin-L 堆叠的比例为 1:1:9:3，由此可以发现 Transformer 网络的第三层的堆叠数量较多，因此ConvNeXt网络依照这个比例将ResNet各阶段的堆叠次数从 (3, 4, 6, 3)调整为(3, 3, 9, 3) ，其比例也保持在 1:1:3:1,改动如下图所示。

<div align=center><img src="./images/stage.png"></div>

这项改动使模型精度提高了0.6%，到达79.4%。

（2）Patchify Stem: Swin-T 网络中的 stem 层为一个卷积核大小为4，步距为4的卷积层，而经典的 ResNet50的stem层是由一个卷积核大小为7，步距为2的卷积层加一个核大小为3，步距为2的最大池化层构成的。因此,ConvNeXt网络将stem层换成了与Swin-T网络相同的卷积核大小为4，步距为4的卷积层。这项改动给模型精度再度带来0.1%的提升，精度达到79.5%。

2. ResNeXt-ify

这一部分中，尝试使用ResNeXt的核心思想--分组卷积，其中为弥补模型容量上的损失增加了网络宽度。同时ConvNext直接让分组数与输入通道数相等，设为96。这样每个卷积核处理一个通道，只在空间维度上做信息混合，获得与自注意力机制类似的效果。这项改动使网络性能再提高1%，达到80.5%。

3. Inverted bottleneck

在Transformer网络中的MLP模块及MobileNet V2中的Inverted Bottleneck模块，都是采用“两头细，中间粗”的反瓶颈结构。因此，ConvNeXt 网络也参照设计了一个类似的 Inverted bottleneck 结构，如下图所示，该过程为从（a）到（b）。

<div align=center><img src="./images/Invert.jpg"></div>

在做完这样的反转之后，虽然depthwise卷积层的FLOPs有所增加，但下采样残差块作用下，整个网络的FLOPs反而被减少，模型精度也提高了0.1%，达到了80.6%。

4. Large kernel size

在经典的CNN网络中我们一般习惯于使用 3×3 的卷积核，Swin Transformer引入了类似卷积核的局部窗口机制，但大小至少有7x7。而 ConvNeXt测试了各种不同尺寸的卷积核，在测试的过程中发现，反转瓶颈层之后放大了卷积层的维度，直接增大卷积核会让参数量显著增加。所以在这之前，还要再做一步操作，在反转瓶颈层的基础上把depthwise卷积层提前，如下图（b）到（c）。

<div align=center><img src="./images/large.jpg"></div>

这项改动暂时将模型精度下降到了79.9%。之后对卷积核大小的试验从3x3到11x11都有尝试，在7x7时模型精度重回80.6%。再往上增加效果则不明显，在ResNet-200上同样如此，最后卷积核大小就定在7x7。

5. Various layer-wise Micro designs

该部分主要将重点放在了激活函数和归一化上，主要进行以下五部分的微观改动。

（1）传统的 CNN 网络中通常使用 RELU 作为网络的激活函数，而目前 Transformer 类型的网络主流上采用 GELU 激活函数，因此 ConvNeXt 网络将 RELU 替换为更常用的 GELU 激活函数。

（2）Swin-T 网络的每一个 Swin Transformer Block 中均只含有一个激活函数，因此受 Swin-T 的启发，ConvNeXt 网络减少了激活函数的使用，每个块只使用一个激活函数，部署在在第二层之后。

（3）与激活函数类似，ConvNeXt 网络也减少了正则化函数的使用，每个块只使用一个正则化函数，部署在第一层之后。

（4）ConvNeXt 不仅减少了正则化函数的使用，还将正则化函数由 BN 替换成 LN。

（5）参考 Swin-T 网络中的 Patch Merging 模块，ConvNeXt 网络单独设计了一个下采样层对特征进行单独的下采样操作。

将以上所有改动汇总起来，ConvNext单个块的结构如下图所示。最终精度达到82.0%，优于Swin-T的81.3%。

<div align=center><img src="./images/Micro.png"></div>

> 本教程将使用ImageNet数据集对ConvNeXt网络进行训练，并对测试结果进行可视化展示。为了节省运行时间，建议用户使用GPU来运行本实验。

## 数据处理

开始实验之前，请确保本地已经安装了Python环境并安装了MindSpore Vision套件。

### 数据准备

在本教程中，我们将使用[ImageNet数据集](https://image-net.org/)，该数据集总共1000个类，每张都是224*224的彩色图像。其中训练集共1,281，167张图像，测试集共50,000张图像。
本案例应用的数据集是ImageNet中筛选出来的子集，运行第一段代码时会自动下载并解压。请确保你的数据集路径如下所示：

```Text
.ImageNet/
    ├── ILSVRC2012_devkit_t12.tar.gz
    ├── train/
    ├── val/
    └── convnext_infer.png
```

In [2]:
import mindspore as ms
from mindspore import context
from mindvision.classification.dataset import ImageNet

# 设置MindSpore运行在图模式，配置运行平台为GPU
context.set_context(mode=context.GRAPH_MODE, device_target='GPU')

# 数据配置参数
data_url = './imagenet2012/'
resize = 224
batch_size = 16

# 加载训练数据集
dataset_train = ImageNet(data_url,
                         split="train",
                         shuffle=True,
                         resize=resize,
                         batch_size=batch_size,
                         repeat_num=1,
                         num_parallel_workers=1).run()

[WARNING] ME(88557:281473649690064,MainProcess):2022-08-15-20:31:31.756.781 [mindspore/dataset/core/config.py:464] The shared memory is on, multiprocessing performance will be improved. Note: the required shared memory can't exceeds 80% of the available shared memory. You can reduce max_rowsize or reduce num_parallel_workers to reduce shared memory usage.


## 构建网络

ConvNeXt主干网络由ConvNeXtBlock模块以及DownSample模块构成，下面将主要介绍这两个网络模块。

### ConvNeXtBlock模块

ConvNeXt Block结构参照了在Transformer网络中的MLP模块及MobileNet V2中的Inverted Bottleneck模块，采用了“两头细，中间粗”的反瓶颈结构。同时，相较于经典的CNN网络使用$3\times3$大小的卷积核和ReLU激活函数，ConvNeXt Block中使用$7\times7$大小的大卷积核以及GELU激活函数。

如下代码定义`ConvNeXtBlock`类实现ConvNeXt Block结构

In [3]:
import numpy as np

import mindspore as ms
from mindspore import nn, ops
from mindspore import Parameter, Tensor

from mindvision.classification.models.blocks import DropPathWithScale


class ConvNeXtBlock(nn.Cell):
    """
    ConvNext Block. There are two equivalent implementations:
    (1) DwConv -> layernorm(channel_first)->1*1 Conv —>GELU -> 1*1 Conv,all in (N, C, H, W);
    (2) DwConv -> Permute to (NHWC), layernorm(channels_last) -> Dense -> GELU -> Dense,
    permute back to (NCHW). We use (2).

    Args:
        dim(int):Number of input channels.
        drop_prob(float): Stochastic depth rate. Default:0.0.
        layer_scale(float): Init value for Layer Scale. Default:1e-6.

    Inputs:
        - **x** (Tensor) - Tensor of shape :math:`(N, C_{in}, H_{in}, W_{in})`.

    Outputs:
        Tensor of shape :math:`(N, C_{out}, H_{out}, W_{out})`.

    Examples:
        >>> ConvNeXtBlock(dim=96, drop_prob=0.0, layer_scale=1e-6)
    """

    def __init__(self,
                 dim: int,
                 drop_prob: float = 0.0,
                 layer_scale: float = 1e-6):
        super(ConvNeXtBlock, self).__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, pad_mode="pad", padding=3, group=dim, has_bias=True)
        self.layer_norm = nn.LayerNorm(normalized_shape=(dim,), epsilon=1e-6)
        self.transpose = ops.Transpose()
        self.pwconv1 = nn.Dense(dim, 4 * dim)
        self.acti = nn.GELU()
        self.pwconv2 = nn.Dense(4 * dim, dim)
        if layer_scale > 0.:
            self.gamma = Parameter(Tensor(layer_scale * np.ones((dim,)), dtype=ms.float32), requires_grad=True)
        else:
            self.gamma = Parameter(Tensor(np.ones((dim,)), dtype=ms.float32), requires_grad=False)
        self.drop_path = DropPathWithScale(drop_prob)

    def construct(self, x):
        """ConvNeXtBlock forward construct"""
        shortcut = x
        x = self.dwconv(x)
        x = self.transpose(x, (0, 2, 3, 1))
        x = self.layer_norm(x)
        x = self.pwconv1(x)
        x = self.acti(x)
        x = self.pwconv2(x)
        x = self.gamma * x
        x = self.transpose(x, (0, 3, 1, 2))
        x = shortcut + self.drop_path(x)
        return x

#### DropPathWithScale层

在上述ConvNeXtBlock中，应用了DropPathWithScale层，该层根据给定的概率`drop_prob`来随机选择网络上数值传递的路径进行drop，可以对整体的模型训练起到防止过拟合的作用，并且参数值根据'keep_prob'进行量化。
如下代码给出`DropPathWithScale`的定义方式

In [4]:
class DropPathWithScale(nn.Cell):
    """
    DropPath function with keep prob scale.

    Args:
        drop_prob(float): Drop rate, (0, 1). Default:0.0
        scale_by_keep(bool): Determine whether to scale. Default: True.

    Inputs:
        - **x** (Tensor) - Tensor of shape :math:`(N, C_{in}, H_{in}, W_{in})`.

    Outputs:
        Tensor of shape :math:`(N, C_{out}, H_{out}, W_{out})`.
    """

    def __init__(self, drop_prob=0.0, scale_by_keep=True):
        super(DropPathWithScale, self).__init__()
        self.drop_prob = drop_prob
        self.keep_prob = 1.0 - self.drop_prob
        if self.keep_prob == 1.0:
            self.keep_prob = 0.9999
        self.scale_by_keep = scale_by_keep
        self.bernoulli = msd.Bernoulli(probs=self.keep_prob)
        self.div = ops.Div()

    def construct(self, x):
        if self.drop_prob > 0.0 and self.training:
            random_tensor = self.bernoulli.sample((x.shape[0],) + (1,) * (x.ndim - 1))
            if self.keep_prob > 0.0 and self.scale_by_keep:
                random_tensor = self.div(random_tensor, self.keep_prob)
            x = x * random_tensor

        return x

#### Transpose层

为了更加方便的进行模型定义，将`mindspore.ops.Transpose`封装为`nn.Cell`类，并且设置参数`target`，用于将输入模型的数据channel维度前置或后置。
如下代码定义`TransposeChannel`类的实现：

In [5]:
from mindspore import ops, nn


class TransposeChannel(nn.Cell):
    """
    Transpose data's channel axis from channel_first(channel_last) to channel_last(channel_first).

    Args:
        target(str): 'channel_first' or 'channel_last'.

    Inputs:
        - **x** (Tensor) - Tensor of shape :math:`(N, C_{in}, H_{in}, W_{in})`.

    Outputs:
        Tensor of shape :math:`(N, C_{out}, H_{out}, W_{out})`.

    Supported Platforms:
        ``Ascend`` ``GPU`` ``CPU``

    Examples:
        >>> transpose = TransposeChannel(target='channel_first')
    """

    def __init__(self, target='channel_first'):
        super(TransposeChannel, self).__init__()
        self.transpose = ops.Transpose()
        if target == 'channel_first':
            self.perm = (0, 3, 1, 2)
        elif target == 'channel_last':
            self.perm = (0, 2, 3, 1)

    def construct(self, x):
        """Transpose layer construct"""
        x = self.transpose(x, self.perm)
        return x

### DownSample模块

DownSample结构图如下图所示，先对输入进行一个LayerNorm，再进行卷积核大小为$2\times2$、`stride`为2的卷积操作。如下代码定义`DownSample`类实现DownSample结构。

In [6]:
from mindspore import nn, ops


class DownSample(nn.Cell):
    """
    Down sample block for ConvNeXt, composed with layer norm and conv2d.

    Args:
        in_channels(int): Number of input channels.
        out_channels(int): Number of output channels.
        kernel_size(int): Convolution kernel size. Default: 2.
        stride(int): stride size. Default: 2.
        eps(float): A value added to the denominator for numerical stability. Default: 1e-6.

    Inputs:
        - **x** (Tensor) - Tensor of shape :math:`(N, C_{in}, H_{in}, W_{in})`.

    Outputs:
        Tensor of shape :math:`(N, C_{out}, H_{out}, W_{out})`.

    Supported Platforms:
        ``GPU``

    Examples:
        >>> DownSample(in_channels=96, out_channels=96, kernel_size=2, stride=2, eps=1e-6)
    """

    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: int = 2,
                 stride: int = 2,
                 eps: float = 1e-6):
        super(DownSample, self).__init__()
        self.transpose = ops.Transpose()
        self.layer_norm = nn.LayerNorm(normalized_shape=(in_channels,), epsilon=eps)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, has_bias=True)

    def construct(self, x):
        """DownSample forward construct"""
        x = self.transpose(x, (0, 2, 3, 1))
        x = self.layer_norm(x)
        x = self.transpose(x, (0, 3, 1, 2))
        x = self.conv(x)
        return x

### 构建ConvNeXt backbone

模型主干网络由4个堆叠的部分组成，`depth`参数控制每部分的`ConvNeXtBlock`个数，`dims`参数控制每部分的特征数目。第i部分由1个`DownSample`模块和`depth[i]`个`ConvNeXtBlock`构成。需要注意的是结构的第一个`DownSample`和`ConvNeXtBlock`与其他有所不同，因此在backbone中单独定义，如下代码定义ConvNeXt的backbone结构：

In [7]:
import mindspore as ms
from mindspore import nn, ops, Tensor

from mindvision.classification.models.blocks import ConvNeXtBlock, DownSample, TransposeChannel


class ConvNeXt(nn.Cell):
    """
    Args:
        in_channels(int): Number of input image channels. Default: 3
        depths (List(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (List(int)): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale (float): Init value for Layer Scale. Default: 1e-6.

    Inputs:
        - **x** (Tensor) - Tensor of shape :math:`(N, C_{in}, H_{in}, W_{in})`.

    Outputs:
        Tensor of shape :math:`(N, C_{out}, H_{out}, W_{out})`.

    Supported Platforms:
        ``GPU``

    Examples:
        >>> backbone = ConvNeXt()
    """

    def __init__(self,
                 in_channels=3,
                 depths=None,
                 dims=None,
                 drop_path_rate=0.,
                 layer_scale=1e-6):
        super(ConvNeXt, self).__init__()
        if not depths:
            depths = [3, 3, 9, 3]
        if not dims:
            dims = [96, 192, 384, 768]
        self.start_cell = nn.SequentialCell([nn.Conv2d(in_channels, dims[0], 4, 4, has_bias=True),
                                             TransposeChannel(target='channel_last'),
                                             nn.LayerNorm(normalized_shape=(dims[0],), epsilon=1e-6),
                                             TransposeChannel(target='channel_first')])
        linspace = ops.LinSpace()
        start = Tensor(0, ms.float32)
        dp_rates = [x.item((0,)) for x in linspace(start, drop_path_rate, sum(depths))]

        self.block1 = nn.SequentialCell([ConvNeXtBlock(dim=dims[0],
                                                       drop_prob=dp_rates[j],
                                                       layer_scale=layer_scale)
                                         for j in range(depths[0])])
        del dp_rates[: depths[0]]

        down_sample_blocks_list = []
        for i in range(3):
            down_sample = DownSample(in_channels=dims[i], out_channels=dims[i+1])
            down_sample_blocks_list.append(down_sample)
            block = nn.SequentialCell([ConvNeXtBlock(dim=dims[i+1],
                                                     drop_prob=dp_rates[j],
                                                     layer_scale=layer_scale)
                                       for j in range(depths[i+1])])
            down_sample_blocks_list.append(block)
            del dp_rates[: depths[i+1]]
        self.down_sample_blocks = nn.SequentialCell(down_sample_blocks_list)

    def construct(self, x):
        x = self.start_cell(x)
        x = self.block1(x)
        x = self.down_sample_blocks(x)
        return x

## 模型实现

### 基本模型

模型主干网络定义完成后，添加neck与head构成ConvNeXt的基础模型结构，模型的neck由一个平均池化层连接一个Layernorm层组成，head检测头为`num_classes=1000`的全连阶层构成，下图为ConvNeXt完整模型结构。

<div align=center><img src="./images/convnext.jpg"></div>

如下代码为ConvNeXt的基本模型构成，通过传入不同的参数来构建不同规格的ConvNeXt网络。

In [8]:
from typing import List

from mindvision.classification.models.backbones import ConvNeXt
from mindvision.classification.models.classifiers import BaseClassifier
from mindvision.classification.models.head import DenseHead
from mindvision.classification.models.neck import AvgPoolingLayerNorm
from mindvision.classification.utils.model_urls import model_urls
from mindvision.utils.load_pretrained_model import LoadPretrainedModel


def _convnext(arch: str,
              depths: List[int],
              dims: List[int],
              pretrained: bool,
              in_channels: int = 3,
              num_classes: int = 1000,
              drop_path_rate: float = 0.,
              layer_scale: float = 1e-6) -> ConvNeXt:
    """ConvNext architecture."""
    backbone = ConvNeXt(
        in_channels=in_channels,
        depths=depths,
        dims=dims,
        drop_path_rate=drop_path_rate,
        layer_scale=layer_scale
    )
    neck = AvgPoolingLayerNorm(num_channels=dims[-1])
    head = DenseHead(input_channel=dims[-1], num_classes=num_classes)
    model = BaseClassifier(backbone, neck, head)

    if pretrained:
        # Download the pre-trained checkpoint file from url, and load
        # checkpoint file.
        LoadPretrainedModel(model, model_urls[arch]).run()

    return model

本案例传入_convnext()函数中depths为[3, 3, 9, 3]，以及dims为[96, 192, 384, 768]，构成tiny规格的convnext_tiny模型，接下来将详细介绍convnext_tiny模型。

### convnext_tiny模型

convnext_tiny网络共有5个卷积结构，一个平均池化层，一个LayerNorm层，一个全连接层，以ImageNet数据集为例：

+ **stem**：输入图片大小为$224\times224$，输入channel为3.首先经过一个卷积核数量为96，卷积核大小为$4\times4$，stride为4的卷积层，接着通过一个LayerNorm层。该层输出feature map大小为$56\times56$，输出channel为96。
+ **res2**：输入feature map大小为$56\times56$，输入channel为96。经过堆叠3个$[d7\times7，96；1\times1，384；1\times1，96]$结构的ConvNeXt Block。该层输出feature map大小为$56\times56$，输出channel为96。
+ **res3**：输入feature map大小为$56\times56$，输入channel为96。经过依次堆叠3个$[d7\times7，192；1\times1，768；1\times1，192]$结构的DownSample块及ConvNeXt Block块。该层输出feature map大小为$28\times28$，输出channel为192。
+ **res4**：输入feature map大小为$28\times28$，输入channel为192。经过依次堆叠9个$[d7\times7，384；1\times1，1536；1\times1，384]$结构的DownSample块及ConvNeXt Block块。该层输出feature map大小为$14\times14$，输出channel为384。
+ **res5**：输入feature map大小为$14\times14$，输入channel为384。经过依次堆叠3个$[d7\times7，768；1\times1，3072；1\times1，768]$结构的DownSample块及ConvNeXt Block块。该层输出feature map大小为$7\times7$，输出channel为768。
+ **average pool & LayerNorm & fc**：输入channel为768，输出channel为分类的类别数。

如下示例代码实现convnext_tiny模型的构建，通过用调函数`convnext_tiny`即可构建convnext_tiny模型：

In [9]:
def convnext_tiny(pretrained: bool = False,
                  in_channels: int = 3,
                  num_classes: int = 1000,
                  drop_path_rate: float = 0.,
                  layer_scale: float = 1e-6
                  ) -> ConvNeXt:
    """
    Constructs a ConvNeXt-tiny architecture.

    Args:
        pretrained(bool): Whether to download and load the pre-trained model. Default: False.
        in_channels(int): Number of input channels.
        num_classes(int): The number of classification. Default: 1000.
        drop_path_rate(float): Stochastic depth rate. Default: 0.
        layer_scale(float): Init value for Layer Scale. Default: 1e-6.

    Inputs:
        - **x**(Tensor) - Tensor of shape: math: `(N, C_{in}, H_{in}, W_{in})`.

    Outputs:
        Tensor of shape: math:`(N, CLASSES_{out})`.

    Supported Platforms:
        ``GPU``

    Examples:
        >>> import numpy as np
        >>>
        >>> import mindspore as ms
        >>> from mindvision.classification.models import convnext_tiny
        >>>
        >>> net = convnext_tiny()
        >>> x = ms.Tensor(np.ones([1, 3, 224, 224]), ms.float32)
        >>> output = net(x)
        >>>print(output.shape)
        (1, 1000)

    About ConvNeXt:

    ConvNeXt pure convolutional neural network is proposed, which is aimed at the very popular swing transformer
    in 2021. Through a series of experimental comparisons, ConvNeXt has faster reasoning speed and higher accuracy
    than swing transformer under the same flops.

    Citation:

    .. code-block::

        @article{,
        title={A ConvNet for the 2020s},
        author={Zhuang, Liu. and Hanzi, Mao. and Chao-Yuan, Wu.},
        journal={},
        year={}
        }
    """
    return _convnext(
        "convnext_tiny", [3, 3, 9, 3], [96, 192, 384, 768],
        pretrained, in_channels, num_classes, drop_path_rate, layer_scale)

## 模型训练

本节调用`convnext_tiny`网络，然后定义`AdamWeightDecay`优化器和`SoftmaxCrossEntropWithLogits`损失函数，通过`model.train`接口对网络进行训练。其中将会打印训练的损失值，并保存评估精度最高的ckpt文件。
以下代码为训练的整体流程，首先调用`convnext_tiny`网络，然后定义优化器和损失函数，通过`model.train`接口对网络进行训练，最后将会打印训练的损失值，并保存评估精度最高的ckpt文件。

### 参数分组

#### 分配参数id

`get_param_id`函数通过`name`传入可训练参数名称，获取对应分组id，id取值为0～13对应14组模型参数分组。每个`DownSample`和`ConvNeXtBlock`为一个单独模块，整个模型中，每隔三个模块分为一组，并分配一个`layer_id`。

In [10]:
def get_param_id(name):
    """
    Get parameter id number from parameter's name.
    The id range comes from 0 to 13.

    Args:
        name: the parameter's name.

    Returns:
        int, the id of the given parameter.
    """

    name_split = name.split('.')

    layer_id = 13
    if name_split[0] == 'backbone':
        if name_split[1] == 'start_cell':
            layer_id = 0
        elif name_split[1] == 'block1':
            layer_id = 1
        elif name_split[1] == 'down_sample_blocks':
            if name_split[2] in ['0', '1']:
                layer_id = 2
            elif name_split[2] == '2':
                layer_id = 3
            elif name_split[2] == '3':
                layer_id = 3 + int(name_split[3]) // 3
            elif name_split[2] in ['4', '5']:
                layer_id = 12

    return layer_id

#### 获取lr scale

初始化assigner时传入长度为14的列表，为14个lr scale数值，`get_lr_scale`方法通过传入`layer_id`参数获取对应的scale。

In [11]:
class ParamLRValueAssigner:
    """
    For given layer_id, get relative lr scale value.

    Args:
        values: param decay values with length 14 for 14 levels

    Returns:
        float, lr scale value for given layer_id
    """

    def __init__(self, values):
        self.values = values

    def get_lr_scale(self, layer_id):
        """
        get lr scale value for given layer_id
        """
        return self.values[layer_id]

#### 参数分组

`get_group`函数通过对不同可训练参数分配`layer_id`来进行参数分组，共14组，相同的id被分配到同一个参数组中并共享同一组超参数。不同分组具备不同scale的初始学习率；参数中，偏置与`gamma`的`weight_decay`值为0。
如函数中传入的`assigner`参数为`None`，则代表不需要lr scale，所有参数初始值均相同。
可选通过传入参数`skip_list`来选择过滤一些参数。

In [12]:
def get_group(network,
              init_lr,
              warmup_epochs,
              epoch_size,
              lr_scheduler,
              step_per_epoch,
              assigner=None,
              weight_decay=1e-5,
              skip_list=None):
    """
    Get parameters group by parameter's name, each group with different learning rate and weight decay value.

    Args:
        network(nn.Cell): The MindSpore network.
        init_lr(float): Init learning rate.
        warmup_epochs(int): Epoch numbers for warmup training.
        epoch_size(int): Total epoch size.
        lr_scheduler: Learning rate schedule function.
        step_per_epoch(int): Number of steps for one training epoch.
        assigner: Learning rate scale assigner function, could be None or ParamLRValueAssigner. Default: None
        weight_decay(float): weight decay init value. Default: 1e-5
        skip_list(List(str)): The list of parameters' names for skipping the group process. Default: None

    Returns:
        List, the groups of network's parameters.
    """

    if skip_list is None:
        skip_list = []
    param_groups = {}

    for (name, param) in network.parameters_and_names():
        if len(param.shape) == 1 or name.endswith('.bias') or name in skip_list:
            group_name = 'no_decay'
            this_weight_decay = 0.
        else:
            group_name = 'decay'
            this_weight_decay = weight_decay

        layer_id = get_param_id(name)
        lr_scale = 1.
        if assigner is not None:
            lr_scale = assigner.get_lr_scale(layer_id)
        group_name = "layer_%d_%s" % (layer_id, group_name)
        if group_name not in param_groups:
            param_groups[group_name] = {'params': [],
                                        'weight_decay': this_weight_decay,
                                        'lr': lr_scheduler(lr=init_lr * lr_scale,
                                                           steps_per_epoch=step_per_epoch,
                                                           warmup_epochs=warmup_epochs,
                                                           max_epoch=epoch_size,
                                                           t_max=150,
                                                           eta_min=0)}

        param_groups[group_name]['params'].append(param)

    return list(param_groups.values())

### 训练

训练模型前，需要先按照论文中给出的参数设置损失函数，优化器以及回调函数，MindSpore Vision套件提供了提供了相应的接口，具体代码如下所示。

In [13]:
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor

from mindvision.engine.lr_schedule.lr_schedule import warmup_cosine_annealing_lr_v1
from mindvision.classification.utils import get_group, ParamLRValueAssigner
from mindvision.classification.models.convnext import convnext_tiny

# 超参数设置
epoch_size = 1
lr = 3e-4
lr_layer_scale = 0.9
step_size = dataset_train.get_dataset_size()

# 搭建网络
network = convnext_tiny()

# 设置学习率
lr_scheduler = warmup_cosine_annealing_lr_v1

# 参数分组
if lr_layer_scale < 1.0:
    num_layers = 12
    lr_scale_values = list(lr_layer_scale ** (num_layers + 1 - i) for i in range(num_layers + 2))
    assigner = ParamLRValueAssigner(lr_scale_values)
else:
    assigner = None
params = get_group(network=network,
                   init_lr=4e-3,
                   warmup_epochs=20,
                   epoch_size=1,
                   lr_scheduler=warmup_cosine_annealing_lr_v1,
                   step_per_epoch=step_size,
                   assigner=assigner,
                   weight_decay=1e-5)

# 定义优化器
network_opt = nn.AdamWeightDecay(params=params,#network.trainable_params(),
                                 learning_rate=lr,
                                 beta1=0.9,
                                 beta2=0.999,
                                 eps=1e-6,
                                 weight_decay=0.0005)

# 定义损失函数
network_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")

# 设置checkpoint
ckpt_config = CheckpointConfig(save_checkpoint_steps=step_size, keep_checkpoint_max=1)
ckpt_callback = ModelCheckpoint(prefix='convnext_tiny', directory='./convnext_tiny', config=ckpt_config)

# 初始化模型
model = ms.Model(network, loss_fn=network_loss, optimizer=network_opt, metrics={"acc"})

# 训练
model.train(epoch_size,
            dataset_train,
            callbacks=[ckpt_callback, LossMonitor()],
            dataset_sink_mode=False)

TypeError: argument of type 'NoneType' is not iterable

```text
Epoch:[0/90], step:[2502/2502], loss:[6.998/6.998], time:1095569.976, lr:0.0049
Epoch time:1106618.839, per step time:445.289, avg loss:6.946
Epoch:[1/90], step:[2502/2502], loss:[6.885/6.885], time:1091877.306, lr:0.0048
Epoch time:1091489.861, per step time:436.539, avg loss:6.885
Epoch:[2/90], step:[2502/2502], loss:[7.026/7.026], time:1093903.073, lr:0.0047
Epoch time:1093905.668, per step time:437.212, avg loss:7.026
```

## 模型评估

使用上述验证精度最高的模型对ImageNet测试数据集进行验证。在此过程中主要应用了Model,ImageNet,convnext_tiny, load_checkpoint, load_param_into_net，SoftmaxCrossEntropyWithLogits等接口。
验证流程大致可以描述为使用convnext_tiny接口定义网络结构，加载ImageNet数据集，并将ckpt文件中的参数加载到定义好的网络结构中，随后设置损失函数，评价指标等等，最后对模型进行编译验证。本教程使用的评价标准为Top_1_Accuracy和Top_5_Accuracy。

In [ ]:
from mindspore import load_checkpoint, load_param_into_net


# 加载验证数据集
dataset_analyse = ImageNet(data_url,
                           split="val",
                           num_parallel_workers=1,
                           resize=resize,
                           batch_size=batch_size)
dataset_eval = dataset_analyse.run()

# 加载模型文件
network = convnext_tiny()
param_dict = load_checkpoint('./convnext_tiny/convnext_tiny-1_125.ckpt')
load_param_into_net(network, param_dict)

# 定义损失函数
network_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")

# 定义评价指标
eval_metrics = {'Top_1_Accuracy': nn.Top1CategoricalAccuracy(),
                'Top_5_Accuracy': nn.Top5CategoricalAccuracy()}

model = ms.Model(network, network_loss, metrics=eval_metrics)

# 评估模型
result = model.eval(dataset_eval)
print(result)

[WARNING] ME(88557:281473649690064,MainProcess):2022-08-15-20:32:27.727.148 [mindspore/dataset/core/config.py:464] The shared memory is on, multiprocessing performance will be improved. Note: the required shared memory can't exceeds 80% of the available shared memory. You can reduce max_rowsize or reduce num_parallel_workers to reduce shared memory usage.


```text
{'Top_1_Accuracy': 0.8255008012820513, 'Top_5_Accuracy': 0.9652644230769231}
```

使用MindSpore Vision套件的ConvNext的Top-1 Accuracy与论文中的对比，如下图所示：

<div align=center><img src="../../../resource/classification/convnext_accuracy.png"></div>

## 模型推理

模型推理过程较为简单，只需使用ImageNet数据集接口读取要推理的图片，加载预训练网络，通过Model.predict方法对图片进行推理即可，具体代码如下所示。

In [ ]:
import numpy as np
from mindspore import context

from mindvision.dataset.download import read_dataset
from mindvision.classification.utils.image import show_result
from mindvision.classification.models.convnext import convnext_tiny
from mindvision.classification.dataset import ImageNet

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

data_path = './ImageNet/'
resize = 224
batch_size = 1

# Data pipeline.
dataset = ImageNet(data_path,
                   split="infer",
                   num_parallel_workers=8,
                   resize=resize,
                   batch_size=batch_size)

dataset_infer = dataset.run()
network = convnext_tiny()

network.set_train(False)

# Init the model.
model = Model(network)

# Begin to infer
image_list, _ = read_dataset(data_path)
for data in dataset_infer.create_dict_iterator(output_numpy=True):
    image = data["image"]
    image = Tensor(image)
    prob = model.predict(image)
    label = np.argmax(prob.asnumpy(), axis=1)
    for i, v in enumerate(label):
        predict = dataset.index2label[v]
        output = {v: predict}
        print(output)
        show_result(img=image_list[i], result=output, out_file=image_list[i])

```text
  {0: 'tench'}
```

推理后的图片如下图所示：
<div align=center><img src="./images/convnext_infer.jpg"></div>

## 总结

本教程实现了一个ConvNeXt模型在ImageNet数据集上进行训练、验证和推理的过程。其中对ConvNeXt模型结构和原理做了简单介绍。

> 如果要详细了解ConvNeXt模型的工作原理，建议对源码进行深层次的阅读，可以参考vision套件:
> https://gitee.com/mindspore/vision/tree/master/examples/classification/convnext

## 引用

[1] Liu Z ,  Mao H ,  Wu C Y , et al. A ConvNet for the 2020s[J]. arXiv e-prints, 2022.